In [1]:
# 9/29/2021 - Building An Equal-Weight S&P 500 Index Fund

# Algorithmic Trading Using Python - Full Course
# https://www.youtube.com/watch?v=xfzGZB4HhEE
# https://iexcloud.io/docs/api/

# https://github.com/nickmccullum/algorithmic-trading-python

# f string

import numpy as np
import pandas as pd
import requests # best python lib for executing HTTP request
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/'
print(api_url)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/


In [10]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
print(api_url)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448


In [11]:
data = requests.get(api_url)
type(data)

requests.models.Response

In [12]:
print(data.status_code)

200


In [13]:
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 86557181, 'calculationPrice': 'iexlasttrade', 'change': 0.989, 'changePercent': 0.007, 'close': 0, 'closeSource': 'iofifcal', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 144.554, 'iexCloseTime': 1678090431912, 'iexLastUpdated': 1702820828852, 'iexMarketPercent': 0.012086828843398734, 'iexOpen': 148.44, 'iexOpenTime': 1706028094269, 'iexRealtimePrice': 149.17, 'iexRealtimeSize': 103, 'iexVolume': 916588, 'lastTradeTime': 1678128387282, 'latestPrice': 149.26, 'latestSource': 'IEX Last Trade', 'latestTime': 'September 29, 2021', 'latestUpdate': 1700832043795, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2455114460516, 'oddLotDelayedPr

In [14]:
print(data['symbol'])

AAPL


In [15]:
price = data['latestPrice']
price

149.26

In [16]:
market_cap = data['marketCap']
market_cap

2455114460516

In [25]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
# final_dataframe = pd.DataFrame([[0,0,0,0]], columns = my_columns)
final_dataframe = pd.DataFrame(columns = my_columns)

In [24]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,0,0,0,0


In [22]:
print(final_dataframe)

   Ticker  Stock Price  Market Capitalization  Number of Shares to Buy
0       0            0                      0                        0


In [35]:
final_dataframe.append(
    pd.Series([symbol, price,market_cap,'N/A'], index = my_columns),
    ignore_index = True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,149.26,2455114460516,N/A


In [41]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ], index = my_columns),
    ignore_index = True
    )

In [42]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,163,50111201913,N/A
1,AAL,21.44,14164749696,N/A
2,AAP,227.11,13687990291,N/A
3,AAPL,144.84,2450214762851,N/A
4,ABBV,108.86,200515793154,N/A
...,...,...,...,...
500,YUM,126.1,36685484684,N/A
501,ZBH,154.26,32076170043,N/A
502,ZBRA,526.07,29322676045,N/A
503,ZION,65.53,10508853027,N/A


In [43]:
def chunks(lst, n): # yields successive n-sized chunks from lst.
    """ yields successive n-sized chunks from lst. """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [44]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [53]:
for i in range(0, len(symbol_groups)):
    print(symbol_groups[i])

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object
100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object
200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object
300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object
400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, dtype: object


In [67]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
#    print(data.status_code)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,167,50216935052,N/A
1,AAL,21.17,13635219847,N/A
2,AAP,217.06,14115075042,N/A
3,AAPL,149.36,2368304498300,N/A
4,ABBV,109.24,193482249974,N/A
...,...,...,...,...
500,YUM,127.03,37204332352,N/A
501,ZBH,152.81,32253701329,N/A
502,ZBRA,551.39,28530434604,N/A
503,ZION,65.67,10427274683,N/A


In [78]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000000


In [79]:
portfolio_size

'10000000'

In [85]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
#    print(final_dataframe.loc[i, 'Number of Shares to Buy'])
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])
final_dataframe    
# number_of_apple_shares = position_size / 500
# print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,167,50216935052,118
1,AAL,21.17,13635219847,935
2,AAP,217.06,14115075042,91
3,AAPL,149.36,2368304498300,132
4,ABBV,109.24,193482249974,181
...,...,...,...,...
500,YUM,127.03,37204332352,155
501,ZBH,152.81,32253701329,129
502,ZBRA,551.39,28530434604,35
503,ZION,65.67,10427274683,301


In [107]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [108]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)


In [105]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

In [109]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [ ]:
writer.save()